# RNNs

## Imports

In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import pickle
from copy import deepcopy
from math import sqrt, ceil
import datetime
import sys
from itertools import product
import pandas as pd
import json
import hyperopt

from data_utils import load_cfar10_batch, load_label_names
from losses import CategoricalHingeLoss, CategoricalCrossEntropyLoss
from activations import LinearActivation, ReLUActivation, SoftmaxActivation, Activation
from initializers import NormalInitializer, XavierInitializer
from layers import Dense, BatchNormalization
from regularizers import L2Regularizer
from models import Model
from metrics import AccuracyMetrics
from optimizers import SGDOptimizer
from lr_schedules import LRConstantSchedule, LRExponentialDecaySchedule, LRCyclingSchedule
from grad_check import eval_numerical_gradient, eval_numerical_gradient_array, numerical_gradient_check_model

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data

In [13]:
class HPData():
    def __init__(self, path_to_file):
        """ Init.
        
        Parameters
        ----------
        path_to_file : str
            Path to text file.
            
        Notes
        -----
        None
        """
        # read text file
        with open(path_to_file, 'r') as f:
            self.book_str = f.read()
        
        # str to chars
        book_data = list(self.book_str)
        # chars to unique chars
        book_chars = list(set(book_data))
        
        # all chars as np
        self.book_data = np.array(book_data)
        # uniqe chars as np
        self.book_chars = np.array(book_chars)
    
    def get_encoder(self,):
        """ Returns encoder, i.e.: unique chars.

        Parameters
        ----------
        None

        Returns
        -------
        book_chars : np.ndarray of shape (n_unique_chars, )
            The encoder as np.

        Notes
        -----
        None
        """
        return self.book_chars
    
    def char_to_idx(self, char):
        """ Convert a char to an index from the encoder np array.

        Parameters
        ----------
        char : str
            A char.

        Returns
        -------
        np.ndarray
            The index repre of char, of shape (,).

        Notes
        -----
        None
        """
        return np.argwhere(char == self.book_chars).flatten()[0]
    
    def idx_to_char(self, idx):
        """ Convert an index to char in the encoder np array.

        Parameters
        ----------
        idx : int
            The index repr of a char.

        Returns
        -------
        str
            The char.

        Notes
        -----
        None
        """
        return self.book_chars[idx]
    
    def encode(self, decoding):
        """ Encode a sequence of chars into a sequence of indices based on the encoder.

        Parameters
        ----------
        chars : np.ndarray
            The sequence of chars, of shape (n_chars,)

        Returns
        -------
        encoding : np.ndarray
            The sequence of index representation of the chars, of shape (n_chars,)

        Notes
        -----
        None
        """
        encoding = []
        
        for d in decoding:
            encoding.append(self.char_to_idx(d))
            
        encoding = np.array(encoding)
        
        return encoding
    
    def decode(self, encoding):
        """ Decode a sequence of indices into a sequence of chars based on the encoder.

        Parameters
        ----------
        encoding : np.ndarray
            The sequence of index representation of the chars, of shape (n_chars,)

        Returns
        -------
        decoding : np.ndarray
            The sequence of chars, of shape (n_chars,)

        Notes
        -----
        None
        """
        decoding = []
        
        for e in encoding:
            decoding.append(self.idx_to_char(e))
            
        decoding = np.array(decoding)
        
        return decoding

In [14]:
class OneHotEncoder():
    def __init__(self, length):
        # length of one-hot encoding
        self.length = length
    
    def __call__(self, x, encode=True):
        """ Encode or decode a sequence x.

        Parameters
        ----------
        x : np.ndarray
            The sequence of index representation of chars, of shape (n_chars,)

        Returns
        -------
        e or d: np.ndarray
            The sequence of one-hot encoded vectors of chars, of shape (n_chars, length)

        Notes
        -----
        None
        """
        if encode:
            e = np.zeros((x.shape[0], self.length))
            e[np.arange(x.shape[0]), x] = 1
            return e.astype(int)
        else:
            d = np.argwhere(one_hot_encoding == 1)[:,1]
            return d.astype(int)

## Read data

Read, encode and decode data.

In [15]:
path_to_file = "data/hp/goblet_book.txt"
hpdata = HPData(path_to_file=path_to_file)
print(hpdata.get_encoder().shape)
print(hpdata.get_encoder())
x = hpdata.book_data[:200]
print(x)
encoding = hpdata.encode(x)
print(hpdata.get_encoder().shape)
print(encoding)
decoding = hpdata.decode(encoding)
print(decoding)

np.testing.assert_array_equal(decoding, x)

(80,)
['f' 'O' '\t' 'n' '(' 'y' ';' 'H' 'm' 'B' '_' '3' 'S' 'o' 'l' ':' 'M' 'Q'
 'Y' '\n' '-' 'p' '1' 'x' 'u' 'C' 'e' '2' 'G' '.' 'Z' ',' '•' 'V' 'X' 'T'
 'D' 'K' 'ü' 'q' 'J' '"' 'g' '?' 'i' 'z' 't' 'c' ')' '7' 'v' 'd' 'L' 'N'
 "'" 'k' '0' '!' 'F' ' ' 'r' 'R' 'U' 'a' 'W' 'h' '}' '9' '4' '6' '/' 'P'
 's' '^' 'j' 'w' 'b' 'I' 'A' 'E']
['H' 'A' 'R' 'R' 'Y' ' ' 'P' 'O' 'T' 'T' 'E' 'R' ' ' 'A' 'N' 'D' ' ' 'T'
 'H' 'E' ' ' 'G' 'O' 'B' 'L' 'E' 'T' ' ' 'O' 'F' ' ' 'F' 'I' 'R' 'E' '\n'
 '\n' 'C' 'H' 'A' 'P' 'T' 'E' 'R' ' ' 'O' 'N' 'E' ' ' '-' ' ' 'T' 'H' 'E'
 ' ' 'R' 'I' 'D' 'D' 'L' 'E' ' ' 'H' 'O' 'U' 'S' 'E' '\n' '\n' '\t' 'T'
 'h' 'e' ' ' 'v' 'i' 'l' 'l' 'a' 'g' 'e' 'r' 's' ' ' 'o' 'f' ' ' 'L' 'i'
 't' 't' 'l' 'e' ' ' 'H' 'a' 'n' 'g' 'l' 'e' 'r' 'o' 'n' ' ' 's' 't' 'i'
 'l' 'l' ' ' 'c' 'a' 'l' 'l' 'e' 'd' ' ' 'i' 't' ' ' '"' 't' 'h' 'e' ' '
 'R' 'i' 'd' 'd' 'l' 'e' ' ' 'H' 'o' 'u' 's' 'e' ',' '"' ' ' 'e' 'v' 'e'
 'n' ' ' 't' 'h' 'o' 'u' 'g' 'h' ' ' 'i' 't' ' ' 'h' 'a' 'd' ' ' 'b' 'e'
 'e' 'n'

## One-ho encode and decode data

In [16]:
onehot_encoder = OneHotEncoder(length=hpdata.get_encoder().size)
one_hot_encoding = onehot_encoder(encoding, encode=True)
print(one_hot_encoding.shape)
one_hot_decoding = onehot_encoder(one_hot_encoding, encode=False)
print(one_hot_decoding.shape)

np.testing.assert_array_equal(one_hot_decoding, encoding)
print(one_hot_decoding[7])
print(one_hot_encoding[7])

print(one_hot_decoding[37])
print(one_hot_encoding[37])

(200, 80)
(200,)
1
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
25
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]


In [17]:
x = np.array([".", "a"])
print(x)
encoding = hpdata.encode(x)
print(hpdata.get_encoder().shape)
print(encoding)
decoding = hpdata.decode(encoding)
print(decoding)

np.testing.assert_array_equal(decoding, x)

one_hot_encoding = onehot_encoder(encoding, encode=True)
print(one_hot_encoding)
print(one_hot_encoding.shape)
np.argwhere(hpdata.get_encoder() == "a")

['.' 'a']
(80,)
[29 63]
['.' 'a']
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0]]
(2, 80)


array([[63]])

## RNN and helpers

In [18]:
class TanhActivation(Activation):
    """ Tanh activation.
    Can be followed by virtually anything.
    Inherits everything from class Activation.

    Attributes
    ----------
    cache : dict
        Run-time cache of attibutes such as gradients.

    Methods
    -------
    __init__()
        Constuctor.
    forward(z)
        Activates the linear transformation of the layer, and
        forward propagates activation. Activation is tanh.
    backward(g)
        Backpropagates incoming gradient into the layer, based on the tanh activation.
    __repr__()
        Returns the string representation of class.
    """

    def __init__(self, ):
        """ Constructor.

        Parameters
        ----------
        None

        Notes
        -----
        None
        """
        super().__init__()

    def forward(self, z):
        """ Activates the linear transformation of the layer, and
        forward propagates activation. Activation is tanh.

        Parameters
        ----------
        z : numpy.ndarray
            Linear transformation of layer.
            Shape is unknown here, but will usually be
            (batch size, this layer output dim = next layer input dim)

        Returns
        -------
        numpy.ndarray
            ReLU activation.

        Notes
        -----
        None
        """
        a = np.tanh(z)
        self.cache["a"] = deepcopy(a)
        return a

    def backward(self, g_in):
        """ Backpropagates incoming gradient into the layer, based on the tanh activation.

        Parameters
        ----------
        g_in : numpy.ndarray
            Incoming gradient to the activation.
            Shape is unknown here, but will usually be
            (batch size, this layer output dim = next layer input dim)

        Returns
        -------
        numpy.ndarray
            Gradient of activation.
            Shape is unknown here, but will usually be
            (batch size, this layer output dim = next layer input dim)

        Notes
        -----
        None
        """
        a = deepcopy(self.cache["a"])
        g_out = (1 - np.power(a, 2)) * g_in
        return g_out

    def __repr__(self):
        """ Returns the string representation of class.

        Parameters
        ----------
        None

        Returns
        -------
        repr_str : str
            The string representation of the class.

        Notes
        -----
        None
        """
        repr_str = "tanh"
        return repr_str

In [19]:
def test_tanh_activation():
    
    tanh_activation = TanhActivation()
    np.random.seed(231)
    x = np.random.randn(5, 10)
    g_in = np.random.randn(*x.shape)
    fx = lambda x: TanhActivation.forward(tanh_activation, x)
    g_out_num = eval_numerical_gradient_array(fx, x, g_in)
    g_out = tanh_activation.backward(g_in)
    np.testing.assert_array_almost_equal(g_out, g_out_num, decimal=6)

    print("test_relu_activation passed")
    
test_tanh_activation()

test_relu_activation passed


In [20]:
class RNN():
    """ Many-to-many."""
    def __init__(self, in_dim, out_dim, hidden_dim, 
                 kernel_h_initializer, bias_h_initializer,
                 kernel_o_initializer, bias_o_initializer,
                 kernel_regularizer, 
                 activation_h, activation_o):
        
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hidden_dim = hidden_dim

        self.kernel_h_initializer = kernel_h_initializer
        self.bias_h_initializer = bias_h_initializer
        self.kernel_o_initializer = kernel_o_initializer
        self.bias_o_initializer = bias_o_initializer

        self.u = kernel_h_initializer.initialize(size=(in_dim, hidden_dim))
        self.w = kernel_h_initializer.initialize(size=(hidden_dim, hidden_dim))
        self.b = bias_h_initializer.initialize(size=(1, hidden_dim))
        
        self.v = kernel_o_initializer.initialize(size=(hidden_dim, out_dim))
        self.c = bias_o_initializer.initialize(size=(1, out_dim))
        
        self.kernel_regularizer = kernel_regularizer

        self.activation_h = activation_h
        self.activation_o = activation_o

        self.cache = {}
        self.grads = {}
        
        self.h_shape = (1, hidden_dim)
        self.cache["h"] = np.zeros(self.h_shape)

        self.has_learnable_params = True
    
    def forward(self, x, **params):
        h = deepcopy(self.cache["h"])
        #h = np.zeros(self.h_shape)
        self.cache["x"] = deepcopy(x)
        #h = np.zeros(self.h_shape)
        h_concat = np.zeros((x.shape[0], h.shape[1]))
        a_concat = np.zeros((x.shape[0], h.shape[1]))
        assert h.shape == (1, self.hidden_dim)
        
        for idx, x_ in enumerate(x):
            x_ = x_.reshape(1,-1)
            assert x_.shape == (1,self.in_dim)
            a = np.dot(x_, self.u) + np.dot(h, self.w) + self.b
            a_concat[idx] = a.reshape(1,-1)
            assert a.shape == (1, self.hidden_dim)
            h = self.activation_h.forward(a)
            #print(self.activation_h.cache["a"].shape)
            h_concat[idx] = deepcopy(h)
            assert h.shape == (1, self.hidden_dim)
        
        # assure good dims for backprop -> only used for 1 vector, so should be ok
        # assure good dims for backprop
        #h_concat_2 = self.activation_h.forward(a_concat)
        #print(self.activation_h.cache["a"].shape)
        #np.testing.assert_array_equal(h_concat, h_concat_2)
        self.cache["h"] = deepcopy(h)
        self.cache["h_concat"] = deepcopy(h_concat)
        self.cache["a_concat"] = deepcopy(a_concat)
        assert h_concat.shape == (x.shape[0], h.shape[1])
        o = np.dot(h_concat, self.v) + self.c
        assert o.shape == (x.shape[0], self.out_dim), f"o.shape={o.shape}"
        p = self.activation_o.forward(o)
        #print(self.activation_o.cache["a"].shape)
        
        assert p.shape == (x.shape[0], self.out_dim)
        return p
    
    def backward(self, g_in, **params):
        # x.shape = (x.shape[0], in_dim)
        x = deepcopy(self.cache["x"])
        # h_concat.shape = (x.shape[0], hidden_dim)
        h_concat = deepcopy(self.cache["h_concat"])
        a_concat = deepcopy(self.cache["a_concat"])
        
        # g_in.shape = (batch_size, )
        assert g_in.shape == (x.shape[0], ), f"g_in.shape={g_in.shape}"
        # g_a_o.shape = (batch_size, out_dim)
        g_a_o = self.activation_o.backward(g_in)
        assert g_a_o.shape == (x.shape[0], self.out_dim)
        
        # g_h_concat.shape = (batch_size, hidden_dim)
        g_h_concat = np.zeros((x.shape[0], self.hidden_dim))
        
        # v.shape = (hidden_dim, out_dim)
        # (1,hidden_dim) = (1,out_dim) * (hidden_dim, out_dim).T
        g_h_concat[-1] = np.dot(g_a_o[-1].reshape(1,-1), self.v.T)
        assert np.dot(g_a_o[-1].reshape(1,-1), self.v.T).shape == (1,self.hidden_dim)
        
        g_a = np.zeros((x.shape[0], self.hidden_dim))
        # (1, hidden_dim) = (1, hidden_dim) * (1, hidden_dim)
        # change cache
        _ = self.activation_h.forward(a_concat[-1].reshape(1,-1))
        g_a[-1] = self.activation_h.backward(g_h_concat[-1]).reshape(1,-1)
        assert self.activation_h.backward(g_h_concat[-1].reshape(1,-1)).shape == (1, self.hidden_dim)
        
        for t in reversed(range(x.shape[0]-1)):
            # (1,hidden_dim) = (1,out_dim) * (hidden_dim, out_dim).T
            # \+ (1,hidden_dim) * (hidden_dim, hidden_dim), maybe w.T?
            g_h_concat[t] = np.dot(g_a_o[t].reshape(1,-1), self.v.T) \
                + np.dot(g_a[t+1].reshape(1,-1), self.w)
            # change cache
            _ = self.activation_h.forward(a_concat[t].reshape(1,-1))
            g_a[t] = self.activation_h.backward(g_h_concat[t])
            assert self.activation_h.backward(g_h_concat[t]).shape == (1, self.hidden_dim)
        
        #print(g_h_concat)
        assert g_h_concat.shape == (x.shape[0], self.hidden_dim)
        assert g_a.shape == (x.shape[0], self.hidden_dim)
        
        # (hidden_dim, out_dim) = (x.shape[0], hidden_dim).T * (x.shape[0], out_dim)
        g_v = np.dot(h_concat.T, g_a_o)
        assert g_v.shape == (self.hidden_dim, self.out_dim)
        self.grads["dv"] = deepcopy(g_v)
        
        # Auxiliar h matrix that includes h_prev
        h_aux = np.zeros(h_concat.shape)
        #h_init = np.zeros((1, self.hidden_dim))
        #h_aux[0, :] = h_init
        h_aux[0] = h_concat[-1].reshape(1,-1)
        h_aux[1:] = h_concat[0:-1]
        assert h_aux.shape == (x.shape[0], self.hidden_dim)
        
        # (hidden_dim, hidden_dim) = (x.shape[0], hidden_dim).T * (x.shape[0], hidden_dim)
        g_w = np.dot(h_aux.T, g_a)
        assert g_w.shape == (self.hidden_dim, self.hidden_dim)
        self.grads["dw"] = deepcopy(g_w)
        
        # (in_dim, hidden_dim) = (x.shape[0], in_dim).T * (x.shape[0], hidden_dim)
        g_u = np.dot(x.T, g_a)
        assert g_u.shape == (self.in_dim, self.hidden_dim)
        self.grads["du"] = deepcopy(g_u)
        
        # (1, hidden_dim) = sum((x.shape[0], self.hidden_dim), axis=0)
        g_b = np.sum(g_a, axis=0).reshape(1,-1)
        assert g_b.shape == (1, self.hidden_dim), f"g_b.shape={g_b.shape}"
        self.grads["db"] = deepcopy(g_b)
        
        # (1, out_dim) = sum((x.shape[0], self.out_dim), axis=0)
        g_c = np.sum(g_a_o, axis=0).reshape(1,-1)
        assert g_c.shape == (1, self.out_dim)
        self.grads["dc"] = deepcopy(g_c)
        
        # compute downstream grad!
        return None
        
    def if_has_learnable_params(self, ):    
        return self.has_learnable_params
    
    def get_u(self, ):
        return deepcopy(self.u)

    def get_w(self, ):
        return deepcopy(self.w)
    
    def get_b(self, ):
        return deepcopy(self.b)
    
    def get_v(self, ):
        return deepcopy(self.v)
    
    def get_c(self, ):
        return deepcopy(self.c)

    def get_learnable_params(self):
        return {
            "u": self.get_u(), "w": self.get_w(), "b": self.get_b(), 
            "v": self.get_v(), "c": self.get_c()
        }
    
    
    def set_u(self, u):
        self.u = deepcopy(u)

    def set_w(self, w):
        self.w = deepcopy(w)
    
    def set_b(self, b):
        self.b = deepcopy(b)
    
    def set_v(self, v):
        self.v = deepcopy(v)
    
    def set_c(self, c):
        self.c = deepcopy(c)

    def set_learnable_params(self, **learnable_params):
        self.set_u(learnable_params["u"])
        self.set_w(learnable_params["w"])
        self.set_b(learnable_params["b"])
        self.set_v(learnable_params["v"])
        self.set_c(learnable_params["c"])

    def get_du(self, ):
        if "du" in self.grads.keys():
            du = self.grads["du"]
            ret = deepcopy(du)
        else:
            ret = None

        return ret
    
    def get_dw(self, ):
        if "dw" in self.grads.keys():
            dw = self.grads["dw"]
            ret = deepcopy(dw)
        else:
            ret = None

        return ret

    def get_db(self, ):
        if "db" in self.grads.keys():
            db = self.grads["db"]
            ret = deepcopy(db)
        else:
            ret = None

        return ret
    
    def get_dv(self, ):
        if "dv" in self.grads.keys():
            dv = self.grads["dv"]
            ret = deepcopy(dv)
        else:
            ret = None

        return ret
    
    def get_dc(self, ):
        if "dc" in self.grads.keys():
            dc = self.grads["dc"]
            ret = deepcopy(dc)
        else:
            ret = None

        return ret

    def get_learnable_params_grads(self):
        return {
            "du": self.get_du(), "dw": self.get_dw(), "db": self.get_db(),
            "dv": self.get_dv(), "dc": self.get_dc()
        }
    
    def if_has_learnable_params(self, ):
        return self.has_learnable_params
        
    def get_reg_loss(self, ):
        return 0.0
    
    def __repr__(self, ):
        repr_str = "rnn: \n" \
                   + f"\t shape -- in: {self.in_dim}, out: {self.out_dim}, hidden: {self.hidden_dim}\n" \
                   + "\t u -- init: " + self.kernel_h_initializer.__repr__() + "\n" \
                    + "\t w -- init: " + self.kernel_h_initializer.__repr__() + "\n" \
                    + "\t b -- init: " + self.bias_h_initializer.__repr__() + "\n" \
                    + "\t v -- init: " + self.kernel_o_initializer.__repr__() + "\n" \
                    + "\t c -- init: " + self.bias_o_initializer.__repr__() + "\n" \
                   + ", reg: " + self.kernel_regularizer.__repr__() + "\n" \
                   + "\t activation: \n \t hidden: " + self.activation_h.__repr__() \
                    + "\t out: " + self.activation_o.__repr__() + "\n"
        return repr_str
    
    
class Synhthetizer():
    def __init__(self, rnn, onehot_encoder):
        self.rnn = rnn
        self.onehot_encoder = onehot_encoder
        self.h_concat = np.zeros(rnn.h_shape)
    
    def sample(self, lenght, p):
        # select character from softmax weighted dist over all chars
        return np.random.choice(range(lenght), size=1, replace=True, p=p.flatten())
        
    
    def __call__(self, ts, init_idx):
        
        x = self.onehot_encoder(np.array([init_idx]).T, encode=True)
        #print(x.shape)
        assert x.shape == (1, self.onehot_encoder.length)
        sequence = []
        
        for t in range(ts):
            p = rnn.forward(x)
            x_idx = self.sample(lenght=x.shape[1], p=p)
            sequence.append(x_idx)
            x = self.onehot_encoder(np.array([x_idx]).T, encode=True)
    
        return np.array(sequence)

### Grad test

Dummy

In [21]:
init_params = {"coeff": 1.0, "mean": 0.0, "std": 0.01}
kernel_h_initializer = NormalInitializer(seed=None, **init_params)
bias_h_initializer = NormalInitializer(seed=None, **init_params)
kernel_o_initializer = NormalInitializer(seed=None, **init_params)
bias_o_initializer = NormalInitializer(seed=None, **init_params)
kernel_regularizer = None

num_inputs = 10
size = (num_inputs, hpdata.get_encoder().size)
x = np.eye(hpdata.get_encoder().size)
x = x[np.random.choice(x.shape[0], size=num_inputs)].astype(int)
y = np.random.randint(hpdata.get_encoder().size, size=num_inputs)

loss = CategoricalCrossEntropyLoss()

rnn = RNN(in_dim=hpdata.get_encoder().size, out_dim=hpdata.get_encoder().size, hidden_dim=5, 
          kernel_h_initializer=kernel_h_initializer, 
          bias_h_initializer=bias_h_initializer, 
          kernel_o_initializer=kernel_o_initializer, 
          bias_o_initializer=bias_o_initializer, 
          kernel_regularizer=kernel_regularizer, 
          activation_h=TanhActivation(),
          activation_o=SoftmaxActivation())

print(rnn)

layers = [rnn]
model = Model(layers)

numerical_gradient_check_model(x, y, model, loss)

rnn: 
	 shape -- in: 80, out: 80, hidden: 5
	 u -- init: normal ~ 1.000000 x N(0.000000, 0.010000^2)
	 w -- init: normal ~ 1.000000 x N(0.000000, 0.010000^2)
	 b -- init: normal ~ 1.000000 x N(0.000000, 0.010000^2)
	 v -- init: normal ~ 1.000000 x N(0.000000, 0.010000^2)
	 c -- init: normal ~ 1.000000 x N(0.000000, 0.010000^2)
, reg: None
	 activation: 
 	 hidden: tanh	 out: softmax

layer=0, param_name=u
max rel error=1.3751199180496758
layer=0, param_name=w
max rel error=0.3311997814964626
layer=0, param_name=b
max rel error=1.0955563435105435
layer=0, param_name=v
max rel error=0.062020333718575564
layer=0, param_name=c
max rel error=1.3892426963187314e-06
test_grad_check passed


real

In [24]:
batch_size = 25
x_chars = hpdata.book_data[:batch_size]
y_chars = hpdata.book_data[1:batch_size+1]
x_encoding = hpdata.encode(x_chars)
y_encoding = hpdata.encode(y_chars)
onehot_encoder = OneHotEncoder(length=hpdata.get_encoder().size)
x_train = onehot_encoder(x_encoding, encode=True)
y_train = y_encoding

init_params = {"coeff": 1.0, "mean": 0.0, "std": 0.01}
kernel_h_initializer = NormalInitializer(seed=None, **init_params)
bias_h_initializer = NormalInitializer(seed=None, **init_params)
kernel_o_initializer = NormalInitializer(seed=None, **init_params)
bias_o_initializer = NormalInitializer(seed=None, **init_params)
kernel_regularizer = None

num_inputs = batch_size

loss = CategoricalCrossEntropyLoss()

rnn = RNN(in_dim=hpdata.get_encoder().size, out_dim=hpdata.get_encoder().size, hidden_dim=5, 
          kernel_h_initializer=kernel_h_initializer, 
          bias_h_initializer=bias_h_initializer, 
          kernel_o_initializer=kernel_o_initializer, 
          bias_o_initializer=bias_o_initializer, 
          kernel_regularizer=kernel_regularizer, 
          activation_h=TanhActivation(),
          activation_o=SoftmaxActivation())

print(rnn)

layers = [rnn]
model = Model(layers)

numerical_gradient_check_model(x_train, y_train, model, loss)

rnn: 
	 shape -- in: 80, out: 80, hidden: 5
	 u -- init: normal ~ 1.000000 x N(0.000000, 0.010000^2)
	 w -- init: normal ~ 1.000000 x N(0.000000, 0.010000^2)
	 b -- init: normal ~ 1.000000 x N(0.000000, 0.010000^2)
	 v -- init: normal ~ 1.000000 x N(0.000000, 0.010000^2)
	 c -- init: normal ~ 1.000000 x N(0.000000, 0.010000^2)
, reg: None
	 activation: 
 	 hidden: tanh	 out: softmax

layer=0, param_name=u
max rel error=1.6851094960056114
layer=0, param_name=w
max rel error=0.26394272299118826
layer=0, param_name=b
max rel error=0.5989718875354738
layer=0, param_name=v
max rel error=0.10395670299550329
layer=0, param_name=c
max rel error=8.106420009714596e-07
test_grad_check passed


### Train

In [32]:
x_chars = hpdata.book_data
y_chars = hpdata.book_data
x_encoding = hpdata.encode(x_chars)
y_encoding = hpdata.encode(y_chars)
onehot_encoder = OneHotEncoder(length=hpdata.get_encoder().size)
x_train = onehot_encoder(x_encoding, encode=True)
#y_train = onehot_encoder(y_encoding, encode=True)
y_train = y_encoding
#print(x_train.shape)
#print(y_train.shape)

In [65]:
print(x_train.shape)
print(y_train.shape)

init_params = {"coeff": 1.0, "mean": 0.0, "std": 0.01}
kernel_h_initializer = NormalInitializer(seed=None, **init_params)
bias_h_initializer = NormalInitializer(seed=None, **init_params)
kernel_o_initializer = NormalInitializer(seed=None, **init_params)
bias_o_initializer = NormalInitializer(seed=None, **init_params)
kernel_regularizer = None

num_inputs = batch_size

loss = CategoricalCrossEntropyLoss()

rnn = RNN(in_dim=hpdata.get_encoder().size, out_dim=hpdata.get_encoder().size, hidden_dim=5, 
          kernel_h_initializer=kernel_h_initializer, 
          bias_h_initializer=bias_h_initializer, 
          kernel_o_initializer=kernel_o_initializer, 
          bias_o_initializer=bias_o_initializer, 
          kernel_regularizer=kernel_regularizer, 
          activation_h=TanhActivation(),
          activation_o=SoftmaxActivation())


loss = CategoricalCrossEntropyLoss()
lr_initial=0.01
optimizer = SGDOptimizer(lr_schedule=LRConstantSchedule(lr_initial))

n_epochs = 5

batch_size = 25
n_batches = int(hpdata.book_data.shape[0] / batch_size)

n_steps = n_epochs * n_batches
n_step = 1

losses_register = []

for n_epoch in range(n_epochs):
    print(f"starting epoch: {n_epoch + 1} ...")
    batches = tqdm(range(n_batches))
    for b in batches:
        batches.set_description(f"batch {b + 1}/{n_batches}")
        x_batch = x_train[b * batch_size:(b + 1) * batch_size]
        y_batch = y_train[b * batch_size + 1:(b + 1) * batch_size + 1]
        y_batch = y_encoding[b * batch_size + 1:(b + 1) * batch_size + 1]

        if y_batch.shape[0] < batch_size:
            continue
        
        scores = rnn.forward(x_batch)
        data_loss = loss.compute_loss(scores, y_batch)
        losses_register.append(data_loss)
        
        params_train = {"mode": "train", "seed": None}
        rnn.backward(loss.grad(), **params_train)

        trainable_params=rnn.get_learnable_params()
        grads=rnn.get_learnable_params_grads()

        for k,v in trainable_params.items():
            trainable_params[k] = deepcopy(v - lr_initial * np.maximum(np.minimum(grads["d"+k], 5), -5))

        rnn.set_learnable_params(**trainable_params)
        if n_step % 1000 == 0:
            print(f"n_step={n_step+1}/{n_steps}, ave loss={np.array(losses_register).sum()/1000}")
            losses_register = []
        n_step += 1

(1107542, 80)
(1107542,)
starting epoch: 1 ...


  0%|          | 0/44301 [00:00<?, ?it/s]

n_step=1001/221505, ave loss=4.12835307074449
n_step=2001/221505, ave loss=3.7219940172493
n_step=3001/221505, ave loss=3.397007991145309
n_step=4001/221505, ave loss=3.255552132025294
n_step=5001/221505, ave loss=3.2232314810375313
n_step=6001/221505, ave loss=3.20077377810724
n_step=7001/221505, ave loss=3.1915159142199316
n_step=8001/221505, ave loss=3.1693290036524764
n_step=9001/221505, ave loss=3.2048821759068913
n_step=10001/221505, ave loss=3.1986789333188335
n_step=11001/221505, ave loss=3.2040383882270747
n_step=12001/221505, ave loss=3.131800934204009
n_step=13001/221505, ave loss=3.1656393523172457
n_step=14001/221505, ave loss=3.1403428957834976
n_step=15001/221505, ave loss=3.170656373607051
n_step=16001/221505, ave loss=3.1352241328982693
n_step=17001/221505, ave loss=3.1509140534615634
n_step=18001/221505, ave loss=3.1426117050731603
n_step=19001/221505, ave loss=3.142112346742293
n_step=20001/221505, ave loss=3.1181409591086453
n_step=21001/221505, ave loss=3.136365010

  0%|          | 0/44301 [00:00<?, ?it/s]

n_step=45001/221505, ave loss=3.1324386696715703
n_step=46001/221505, ave loss=3.0906281657317147
n_step=47001/221505, ave loss=3.1312304273997893
n_step=48001/221505, ave loss=3.1347714973370264
n_step=49001/221505, ave loss=3.157347300453797
n_step=50001/221505, ave loss=3.1484843090629364
n_step=51001/221505, ave loss=3.1561993349642106
n_step=52001/221505, ave loss=3.1383403254551587
n_step=53001/221505, ave loss=3.1687653954891237
n_step=54001/221505, ave loss=3.181737846945337
n_step=55001/221505, ave loss=3.184233494690813
n_step=56001/221505, ave loss=3.13768514285809
n_step=57001/221505, ave loss=3.1546897030977608
n_step=58001/221505, ave loss=3.136291238575596
n_step=59001/221505, ave loss=3.1499285090106777
n_step=60001/221505, ave loss=3.1256369086844935
n_step=61001/221505, ave loss=3.1434352507687837
n_step=62001/221505, ave loss=3.130611023445767
n_step=63001/221505, ave loss=3.142933711148758
n_step=64001/221505, ave loss=3.11061139279663
n_step=65001/221505, ave loss=

...

n_step=89001/221505, ave loss=3.1204819089492517
n_step=90001/221505, ave loss=3.117434469105209
n_step=91001/221505, ave loss=3.0977950662836267
n_step=92001/221505, ave loss=3.1513816173319884
n_step=93001/221505, ave loss=3.132502871263346
n_step=94001/221505, ave loss=3.134323289100029
n_step=95001/221505, ave loss=3.1386393286899366
n_step=96001/221505, ave loss=3.1082997247335835
n_step=97001/221505, ave loss=3.116812746799676
n_step=98001/221505, ave loss=3.1140673890151307
n_step=99001/221505, ave loss=3.100661417938226
n_step=100001/221505, ave loss=3.074824954513689
n_step=101001/221505, ave loss=3.0602393134853436
n_step=102001/221505, ave loss=3.036621234094847
n_step=103001/221505, ave loss=3.0368167958528067
n_step=104001/221505, ave loss=2.9974018560785036
n_step=105001/221505, ave loss=2.99815608849805
n_step=106001/221505, ave loss=3.025134303230099
n_step=107001/221505, ave loss=3.003989920737431
n_step=108001/221505, ave loss=2.9653547652047827
n_step=109001/221505, 

  0%|          | 0/44301 [00:00<?, ?it/s]

n_step=133001/221505, ave loss=2.8455296501468403
n_step=134001/221505, ave loss=2.821587146914487
n_step=135001/221505, ave loss=2.809090089870805
n_step=136001/221505, ave loss=2.8333809814430846
n_step=137001/221505, ave loss=2.8304250755005467
n_step=138001/221505, ave loss=2.830107837400631
n_step=139001/221505, ave loss=2.858385791890447
n_step=140001/221505, ave loss=2.835754696702159
n_step=141001/221505, ave loss=2.7979053744614637
n_step=142001/221505, ave loss=2.848945606916767
n_step=143001/221505, ave loss=2.85514287184394
n_step=144001/221505, ave loss=2.8583563208690923
n_step=145001/221505, ave loss=2.8147744360921205
n_step=146001/221505, ave loss=2.8431243164182396
n_step=147001/221505, ave loss=2.8415830730619307
n_step=148001/221505, ave loss=2.8003046056240595
n_step=149001/221505, ave loss=2.7732139879521522
n_step=150001/221505, ave loss=2.815691369739401
n_step=151001/221505, ave loss=2.7881254733331136
n_step=152001/221505, ave loss=2.785407438332763
n_step=153

  0%|          | 0/44301 [00:00<?, ?it/s]

n_step=178001/221505, ave loss=2.658030061588818
n_step=179001/221505, ave loss=2.6565650226338238
n_step=180001/221505, ave loss=2.6667380509711767
n_step=181001/221505, ave loss=2.6720977315641474
n_step=182001/221505, ave loss=2.682319949845126
n_step=183001/221505, ave loss=2.6981037499674363
n_step=184001/221505, ave loss=2.7107980534714518
n_step=185001/221505, ave loss=2.642804827559416
n_step=186001/221505, ave loss=2.69316147061966
n_step=187001/221505, ave loss=2.6935088216749996
n_step=188001/221505, ave loss=2.7306860148230174
n_step=189001/221505, ave loss=2.6728529204559655
n_step=190001/221505, ave loss=2.685043001326034
n_step=191001/221505, ave loss=2.644592998485159
n_step=192001/221505, ave loss=2.686162035008383
n_step=193001/221505, ave loss=2.646573811774729
n_step=194001/221505, ave loss=2.668266128013147
n_step=195001/221505, ave loss=2.6380477015038823
n_step=196001/221505, ave loss=2.666942914837526
n_step=197001/221505, ave loss=2.6130014393243144
n_step=1980

In [67]:
synhthetizer = Synhthetizer(rnn, onehot_encoder)
sequence= synhthetizer(ts=2000, init_idx=1)
print("".join(hpdata.decode(sequence.flatten())))

hauneend  fioneys Hol,".
"I  fol, --M..hid thiponed 'aul Homlmr wlabs. baancede any
-pemedaumen bo, weus gv gupl ore sd choh the waheeet.  wev anull," heme sondins Soyd.  Dooc tf  wory ahe ."".Wo inile thep told tholg acd "2hot indtw Mes,at teid aurr the? t welly.  seouk oH lat ird sere tat shadarcehevd wond het'fud ag his maedg, foft.  DonysRarled wiMg. hod hiret to, we he ton, bandilire- Pel.  "9ou Mokd teg miidtianr wat bh to fhrinteas ad me thh!.hee salraus  hois tha p"jocd the.  un thaiindseye jih bomleps mrasrtame cam pos ors beSmrd s4gatnp de, cein, d."heeloink lainid rgiwicm ..  P	oord fpf, Sircugt  mind alear feme beisorels ghom NofC woll hesem uTpals leordpe saile.. Hindpercepe.

Q)cto ofe sglr.
"uhe, wel. Mhipine bcunt," ""Cat pssn pinVd I wuud, "Ooous Iins.  wad .
'Bopy kepwindey xoh  ""Nheimd elalnrisdras ihdsre,  se Ie.  con, Tfar bot sapceint pictpcy the hot .he any sole chewarte? homdonds Rasrreld seemy theud patry cadhe Oatcpe talizerappiwlet lofeh, tuf leireoll chih M

In [39]:
lol = np.array([[1,4,40],[-10,2,0]])
np.maximum(np.minimum(lol, 5), -5)

array([[ 1,  4,  5],
       [-5,  2,  0]])